<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Audit-Your-Customer-Journey-Analytics-Implementation-with-Python" data-toc-modified-id="Audit-Your-Customer-Journey-Analytics-Implementation-with-Python-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Audit Your Customer Journey Analytics Implementation with Python</a></span><ul class="toc-item"><li><span><a href="#What-the-Script-Does" data-toc-modified-id="What-the-Script-Does-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>What the Script Does</a></span></li><li><span><a href="#Pre-requisites" data-toc-modified-id="Pre-requisites-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Pre-requisites</a></span></li><li><span><a href="#Usage" data-toc-modified-id="Usage-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Usage</a></span></li><li><span><a href="#Use-Cases" data-toc-modified-id="Use-Cases-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Use Cases</a></span></li></ul></li></ul></div>

## Audit Your Customer Journey Analytics Implementation with Python

This script is designed to help you audit your Customer Journey Analytics (CJA) implementation by generating a comprehensive spreadsheet. This spreadsheet will list all metrics and dimensions available in your selected CJA Data View, providing an invaluable resource akin to a Solution Design Reference (SDR) document in traditional Adobe Analytics. This tool is especially useful for identifying what's been configured in CJA and ensuring your analytics setup aligns with your measurement strategy.

### What the Script Does

1. **Initial Setup**: Utilizes the `cjapy` library to interact with your CJA account, requiring a configuration file for authentication.
2. **Data View Selection**: Targets a specific CJA Data View by its ID for auditing.
3. **Retrieves Metrics and Dimensions**: Fetches all metrics and dimensions configured in the selected Data View.
4. **Generates a Detailed Spreadsheet**: Outputs an Excel file containing:
    - A sheet with Metadata for the Data View.
    - A sheet with Data View details.
    - A sheet listing all Metrics, formatted for readability.
    - A sheet listing all Dimensions, also formatted for readability.

### Pre-requisites

- Python installed on your system.
- The `cjapy` library installed. You can install it using `pip install cjapy`.
- An existing `myconfig.json` configuration file with your CJA credentials.
- Pandas library installed for data manipulation and Excel file generation.

### Usage

1. Ensure you have your `myconfig.json` file ready and correctly configured.
2. Replace `"dv_677ea9291244fd082f02dd42"` with your actual Data View ID.
3. Run the script. It will automatically generate an Excel file named `CJA_DataView_[DataViewName]_[DataViewID]_SDR.xlsx`.
4. Open the generated Excel file to review your CJA implementation details.

### Use Cases

- **Comprehensive Audit**: Quickly understand the breadth and depth of your CJA setup, including what metrics and dimensions are available for analysis.
- **Implementation Verification**: Ensure that your CJA implementation matches your initial planning documents or SDR.
- **Onboarding**: Assist new team members in getting up to speed with the existing CJA setup by providing them with a detailed view of the data structure.
- **Documentation**: Automatically generate part of your project documentation that outlines the analytical capabilities enabled in CJA.

This script automates what could be a time-consuming manual process, allowing you to spend more time on analysis and less on administration.

In [ ]:
import cjapy
import pandas as pd
import json
from datetime import datetime
import pytz

# Instantiate cjapy
cjapy.importConfigFile("myconfig.json")
cja = cjapy.CJA()

# Set the Data View id we want into a variable
data_view = "dv_677ea9291244fd082f02dd42"

# Grab all available CJA metrics and dimensions
metrics = cja.getMetrics(data_view, inclType=True, full=True)
dimensions = cja.getDimensions(data_view, inclType=True, full=True)
lookup_data = cja.getDataView(data_view)

# Process lookup data into DataFrame
lookup_data = {k: [v] if not isinstance(v, (list, tuple)) else v for k, v in lookup_data.items()}
max_length = max(len(v) for v in lookup_data.values())
lookup_data = {k: v + [None] * (max_length - len(v)) for k, v in lookup_data.items()}
lookup_df = pd.DataFrame(lookup_data)

# Enhanced metadata creation
# Calculate metrics summary
metric_types = metrics['type'].value_counts().to_dict()
metric_summary = [f"{type_}: {count}" for type_, count in metric_types.items()]

# Calculate dimensions summary
dimension_types = dimensions['type'].value_counts().to_dict()
dimension_summary = [f"{type_}: {count}" for type_, count in dimension_types.items()]

# Get current timezone and formatted timestamp
local_tz = datetime.now().astimezone().tzinfo
current_time = datetime.now(local_tz)
formatted_timestamp = current_time.strftime('%Y-%m-%d %H:%M:%S %Z')

# Create enhanced metadata DataFrame
metadata_df = pd.DataFrame({
    'Property': [
        'Generated Date & timestamp and timezone',
        'Data View ID',
        'Data View Name',
        'Total Metrics',
        'Metrics Breakdown',
        'Total Dimensions',
        'Dimensions Breakdown'
    ],
    'Value': [
        formatted_timestamp,
        data_view,
        lookup_data["name"][0],
        len(metrics),
        '\n'.join(metric_summary),
        len(dimensions),
        '\n'.join(dimension_summary)
    ]
})

# Function to format JSON cells
def format_json_cell(value):
    if isinstance(value, (dict, list)):
        return json.dumps(value, indent=2)
    return value

# Apply JSON formatting to all dataframes
for col in lookup_df.columns:
    lookup_df[col] = lookup_df[col].map(format_json_cell)
for col in metrics.columns:
    metrics[col] = metrics[col].map(format_json_cell)
for col in dimensions.columns:
    dimensions[col] = dimensions[col].map(format_json_cell)

# Create Excel file name
excel_file_name = f'CJA_DataView_{lookup_data["name"][0]}_{data_view}_SDR.xlsx'

def apply_excel_formatting(writer, df, sheet_name):
    # Write dataframe to sheet
    df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    
    # Add formats
    header_format = workbook.add_format({
        'bold': True,
        'bg_color': '#366092',
        'font_color': 'white',
        'border': 1,
        'align': 'center',
        'text_wrap': True
    })
    
    grey_format = workbook.add_format({
        'bg_color': '#F2F2F2',
        'border': 1,
        'text_wrap': True,
        'align': 'top',
        'valign': 'top'
    })
    
    white_format = workbook.add_format({
        'bg_color': '#FFFFFF',
        'border': 1,
        'text_wrap': True,
        'align': 'top',
        'valign': 'top'
    })
    
    # Format header row
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value, header_format)
    
    # Set row height and column width with text wrapping
    max_column_width = 100  # Maximum width in characters
    for idx, col in enumerate(df.columns):
        series = df[col]
        max_len = min(
            max(
                max(len(str(val).split('\n')[0]) for val in series),
                len(str(series.name))
            ) + 2,
            max_column_width
        )
        worksheet.set_column(idx, idx, max_len)
    
    # Apply alternating row colors and set row height
    for idx in range(len(df)):
        # Calculate appropriate row height based on content
        max_lines = max(str(val).count('\n') for val in df.iloc[idx]) + 1
        row_height = min(max_lines * 15, 400)  # Limit maximum row height
        worksheet.set_row(
            idx + 1,
            row_height,
            grey_format if idx % 2 == 0 else white_format
        )

# Write to Excel with formatting
with pd.ExcelWriter(excel_file_name, engine='xlsxwriter') as writer:
    for sheet_data in [
        (metadata_df, 'Metadata'),
        (lookup_df, 'DataView'),
        (metrics, 'Metrics'),
        (dimensions, 'Dimensions')
    ]:
        apply_excel_formatting(writer, sheet_data[0], sheet_data[1])